In [1]:
import pandas as pd
import numpy as np

### Get size of tree database

The tree database has 683,788 rows.

In [2]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=' + str(0) +\
        '&$select=count(tree_id)').replace(' ', '%20')
soql_size = pd.read_json(soql_url)
soql_size

,count_tree_id
0,683788


### General Introduction on Data Retriving 

This assignment is based on New York City tree census data which is availble at https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh.  From the existing 600,000 rows in the  data set I will be retriving only relevant columns and relevant aggregation through the API call. 

Explanation of the Columns retrieved: 

- borocode: 1 (Manhattan), 2 (Bronx), 3 (Brooklyn), 4 (Queens), 5 (Staten Island)
- spc_common: specie name (132 unique species)
- health: good, fair, poor
- steward: Indicates the number of unique signs of stewardship observed for this tree (none, 1or2, 3or4, 4ormore)

In average I have identfied  4565 rows by grouping the data in borocode, spc_common, health, and steward. I have set the limit value to 1000 for ease of analyzing .I have also set  offset value increments by 1000 and  max_row to be 5000. For the sake of avpinding error I have 'borocode' instead of 'borough'.

In [13]:
offset = 1000
max_row = 5000

In [38]:
for x in range(0, max_row, offset):
    #print('x is ' + str(x))
    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=1000&$offset=' + str(x) +\
        '&$select=borocode,spc_common,health,steward,count(tree_id)' +\
        '&$group=borocode,spc_common,health,steward').replace(' ', '%20')
    soql_trees = pd.read_json(soql_url)
    if(x==0):
        df = pd.DataFrame(columns=list(soql_trees.columns.values))
    df = df.append(soql_trees)
    #print(df)

In [39]:
df = df.reset_index(drop=True)

Total retrieved data set: 

In [40]:
len(df)

4565

I have found  133 unique tree specie name

In [41]:
len(list(df.spc_common.unique()))

133

Remove rows that do not have complete data.  

In [42]:
df = df.dropna(axis=0, how='any')

In [43]:
#df.to_csv('tree_data.csv')

In [44]:
df.head(5)

,borocode,count_tree_id,health,spc_common,steward
0,3,644,Good,London planetree,3or4
1,3,16,Good,Kentucky coffeetree,4orMore
2,3,41,Poor,sycamore maple,None
3,5,1,Poor,Chinese tree lilac,1or2
4,5,5094,Good,red maple,None


---

### Data Preparation 

Based of the prepartion of the data arborists will be equiped with relevant information to answer the following questions.

1. What proportion of trees are in good, fair, or poor health according to the ‘health’variable?
2. Is there any impact on the health of trees caused by the stewards, which is measured by 'steward variable '?

### What proportion of trees are in good, fair, or poor health according to the ‘health’variable?

The application will allow arborist to select one specie, and the application will display proportion of trees that are in good, fair, or poor health across all boroughs. Arborist will be able to compare health of particular specie across all five boroughs. I will use bar graphs to present the proportions grouped by broroughs for each health status. 



In [127]:
df_totals = df.groupby(['borocode', 'spc_common'])['count_tree_id'].sum()
df_total_by_borocode_specie_health = df.groupby(['borocode', 'spc_common', 'health'])['count_tree_id'].sum()

In [128]:
df_totals.head(5)

borocode  spc_common            
1         'Schubert' chokecherry     163
          American beech              22
          American elm              1698
          American hophornbeam        84
          American hornbeam           85
Name: count_tree_id, dtype: int64

In [129]:
df_total_by_borocode_specie_health.head(5)

borocode  spc_common              health
1         'Schubert' chokecherry  Fair       40
                                  Good      111
                                  Poor       12
          American beech          Fair        4
                                  Good       15
Name: count_tree_id, dtype: int64

In [130]:
df_totals = df_totals.reset_index(drop=False)
df_total_by_borocode_specie_health = df_total_by_borocode_specie_health.reset_index(drop=False)

In [131]:
df_totals.head(5)

,borocode,spc_common,count_tree_id
0,1,'Schubert' chokecherry,163
1,1,American beech,22
2,1,American elm,1698
3,1,American hophornbeam,84
4,1,American hornbeam,85


In [132]:
df_total_by_borocode_specie_health.head(5)

,borocode,spc_common,health,count_tree_id
0,1,'Schubert' chokecherry,Fair,40
1,1,'Schubert' chokecherry,Good,111
2,1,'Schubert' chokecherry,Poor,12
3,1,American beech,Fair,4
4,1,American beech,Good,15


In [136]:
df_totals.columns = ['borocode', 'spc_common', 'total_for_specie_in_borough']
df_total_by_borocode_specie_health.columns = ['borocode', 'spc_common', 'health', 'total']

In [141]:
tree_proportions = pd.merge(df_total_by_borocode_specie_health, df_totals, on=['borocode', 'spc_common'])

In [144]:
tree_proportions.head(5)

,borocode,spc_common,health,total,total_for_specie_in_borough
0,1,'Schubert' chokecherry,Fair,40,163
1,1,'Schubert' chokecherry,Good,111,163
2,1,'Schubert' chokecherry,Poor,12,163
3,1,American beech,Fair,4,22
4,1,American beech,Good,15,22


In [145]:
tree_proportions['ratio'] = tree_proportions['total']/ tree_proportions['total_for_specie_in_borough']

In [148]:
tree_proportions.head(10)

,borocode,spc_common,health,total,total_for_specie_in_borough,ratio
0,1,'Schubert' chokecherry,Fair,40,163,0.245399
1,1,'Schubert' chokecherry,Good,111,163,0.680982
2,1,'Schubert' chokecherry,Poor,12,163,0.073620
3,1,American beech,Fair,4,22,0.181818
4,1,American beech,Good,15,22,0.681818
5,1,American beech,Poor,3,22,0.136364
6,1,American elm,Fair,259,1698,0.152532
7,1,American elm,Good,1361,1698,0.801531
8,1,American elm,Poor,78,1698,0.045936
9,1,American hophornbeam,Fair,12,84,0.142857


----

###  Is there any impact on the health of trees caused by the stewards, which is measured by 'steward variable '?

Scatter plot will be used to represent the overall health status of the selected specie across all the boroughs which will be  determined by assigning a numeric value to each health level (Poor=1, Fair=2, Good=3) and then calculating a weighted average for the selected specie for each borough. The overall health index score has a minimum score of 1 and a maximum score of 3. 



In [153]:
list(df['steward'].unique())

['3or4', '4orMore', 'None', '1or2']

In [154]:
df.head(10)

,borocode,count_tree_id,health,spc_common,steward
0,3,644,Good,London planetree,3or4
1,3,16,Good,Kentucky coffeetree,4orMore
2,3,41,Poor,sycamore maple,None
3,5,1,Poor,Chinese tree lilac,1or2
4,5,5094,Good,red maple,None
5,5,6,Fair,boxelder,None
6,4,10,Good,Atlantic white cedar,3or4
7,2,3,Fair,purple-leaf plum,3or4
8,2,25,Poor,'Schubert' chokecherry,None
9,2,16,Fair,hawthorn,1or2


In [182]:
df.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)

,borocode,count_tree_id,health,spc_common,steward
620,1,56,Good,'Schubert' chokecherry,1or2
2510,1,11,Poor,'Schubert' chokecherry,1or2
3561,1,23,Fair,'Schubert' chokecherry,1or2
1851,1,21,Good,'Schubert' chokecherry,3or4
2353,1,2,Fair,'Schubert' chokecherry,3or4
434,1,3,Good,'Schubert' chokecherry,4orMore
1422,1,31,Good,'Schubert' chokecherry,None
3342,1,15,Fair,'Schubert' chokecherry,None
4085,1,1,Poor,'Schubert' chokecherry,None
912,1,7,Good,American beech,1or2


In [193]:
df_total_by_steward = df.groupby(['borocode', 'spc_common', 'steward'])['count_tree_id'].sum()
df_total_by_steward = df_total_by_steward.reset_index(drop=False)
df_total_by_steward.columns = ['borocode', 'spc_common', 'steward', 'steward_total']
df_total_by_steward.head(10)

,borocode,spc_common,steward,steward_total
0,1,'Schubert' chokecherry,1or2,90
1,1,'Schubert' chokecherry,3or4,23
2,1,'Schubert' chokecherry,4orMore,3
3,1,'Schubert' chokecherry,None,47
4,1,American beech,1or2,10
5,1,American beech,None,12
6,1,American elm,1or2,495
7,1,American elm,3or4,102
8,1,American elm,4orMore,15
9,1,American elm,None,1086


In [194]:
df_steward = pd.merge(df, df_total_by_steward, on=['borocode', 'spc_common', 'steward'])
df_steward.head(10)

,borocode,count_tree_id,health,spc_common,steward,steward_total
0,3,644,Good,London planetree,3or4,772
1,3,18,Poor,London planetree,3or4,772
2,3,110,Fair,London planetree,3or4,772
3,3,16,Good,Kentucky coffeetree,4orMore,18
4,3,2,Fair,Kentucky coffeetree,4orMore,18
5,3,41,Poor,sycamore maple,None,418
6,3,75,Fair,sycamore maple,None,418
7,3,302,Good,sycamore maple,None,418
8,5,1,Poor,Chinese tree lilac,1or2,32
9,5,7,Fair,Chinese tree lilac,1or2,32


In [198]:
di = {'Poor':1, 'Fair':2, 'Good':3}
df_steward['health_level'] = df_steward['health'].map(di)
df_steward.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)
df_steward['health_index'] = (df_steward['count_tree_id']/df_steward['steward_total']) * df_steward['health_level']
df_steward.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)
df_overall_health_index = df_steward.groupby(['borocode', 'spc_common', 'steward'])['health_index'].sum()
df_overall_health_index = df_overall_health_index.reset_index(drop=False)
df_overall_health_index.columns = ['borocode', 'spc_common', 'steward', 'overall_health_index']
di2 = {'3or4':3, '4orMore':4, 'None':1, '1or2':2}
df_overall_health_index['steward_level'] = df_overall_health_index['steward'].map(di2)
di3 = { 1:'Manhattan', 2:'Bronx', 3:'Brooklyn', 4:'Queens', 5:'Staten Island'}
df_overall_health_index['borough'] = df_overall_health_index['borocode'].map(di3)
df_overall_health_index['spc_common'] = df_overall_health_index['spc_common'].apply(lambda x: x.title())
df_overall_health_index.head(10)

,borocode,spc_common,steward,overall_health_index,steward_level,borough
0,1,'Schubert' Chokecherry,1or2,2.500000,2,Manhattan
1,1,'Schubert' Chokecherry,3or4,2.913043,3,Manhattan
2,1,'Schubert' Chokecherry,4orMore,3.000000,4,Manhattan
3,1,'Schubert' Chokecherry,None,2.638298,1,Manhattan
4,1,American Beech,1or2,2.700000,2,Manhattan
5,1,American Beech,None,2.416667,1,Manhattan
6,1,American Elm,1or2,2.707071,2,Manhattan
7,1,American Elm,3or4,2.686275,3,Manhattan
8,1,American Elm,4orMore,2.800000,4,Manhattan
9,1,American Elm,None,2.783610,1,Manhattan
